In [53]:
import argparse,os,logging,psutil,time
from joblib import Parallel,delayed

#import utils file
%run utils.ipynb
%run make_graph2vec_corpus.ipynb
%run train_utils.ipynb
%run classify.ipynb

logger = logging.getLogger()
logger.setLevel("INFO")

In [2]:
args = argparse.ArgumentParser("graph2vec")
    
args.add_argument("-c","--corpus", default = "../data/kdd_datasets/ptc",
                  help="Path to directory containing graph files to be used for graph classification or clustering")

args.add_argument('-l','--class_labels_file_name', default='../data/kdd_datasets/ptc.Labels',
                  help='File name containg the name of the sample and the class labels')

args.add_argument('-o', "--output_dir", default = "../embeddings",
                  help="Path to directory for storing output embeddings")

args.add_argument('-b',"--batch_size", default=128, type=int,
                  help="Number of samples per training batch")

args.add_argument('-e',"--epochs", default=1000, type=int,
                  help="Number of iterations the whole dataset of graphs is traversed")

args.add_argument('-d',"--embedding_size", default=1024, type=int,
                  help="Intended graph embedding size to be learnt")

args.add_argument('-neg', "--num_negsample", default=10, type=int,
                  help="Number of negative samples to be used for training")

args.add_argument('-lr', "--learning_rate", default=0.3, type=float,
                  help="Learning rate to optimize the loss function")

args.add_argument("--wlk_h", default=3, type=int, help="Height of WL kernel (i.e., degree of rooted subgraph "
                                                       "features to be considered for representation learning)")

args.add_argument('-lf', '--label_filed_name', default='Label', help='Label field to be used '
                                                                     'for coloring nodes in graphs using WL kenrel')

#args.parse_args()

_StoreAction(option_strings=['-lf', '--label_filed_name'], dest='label_filed_name', nargs=None, const=None, default='Label', type=None, choices=None, help='Label field to be used for coloring nodes in graphs using WL kenrel', metavar=None)

In [7]:
# Set it manually for now
corpus_dir = "/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc"
output_dir = "/home/ipsita/BTP/graph2vec/embeddings"
batch_size = 128
epochs = 100
embedding_size = 1024
num_negsample = 10
learning_rate = 0.3
wlk_h = 3
label_filed_name = 'Label'
class_labels_fname = '/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc.Labels'

wl_extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

assert os.path.exists(corpus_dir), "File {} does not exist".format(corpus_dir)
assert os.path.exists(output_dir), "Dir {} does not exist".format(output_dir)

In [8]:
graph_files = get_files(dirname=corpus_dir, extn='.gexf', max_files=0)
logging.info('Loaded {} graph file names form {}'.format(len(graph_files),corpus_dir))

INFO:root:Loaded 344 graph file names form /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc


In [9]:
t0 = time()
wlk_relabel_and_dump_memory_version(graph_files, max_h=wlk_h, node_label_attr_name=label_filed_name)
logging.info('dumped sg2vec sentences in {} sec.'.format(time() - t0))

loaded all graphs in 0.33 sec
initial relabeling done in 0.06 sec
WL iteration 1 done in 0.06 sec.
num of WL rooted subgraphs in iter 1 is 160
WL iteration 2 done in 0.07 sec.
num of WL rooted subgraphs in iter 2 is 1038


INFO:root:dumped sg2vec sentences in 0.6182849407196045 sec.


WL iteration 3 done in 0.09 sec.
num of WL rooted subgraphs in iter 3 is 2624
dumped sg2vec sentences in 0.0


In [22]:
t0 = time()
embedding_fname = train_skipgram(corpus_dir, wl_extn, learning_rate, embedding_size, num_negsample,
                                 epochs, batch_size, output_dir)
logging.info('Trained the skipgram model in {} sec.'.format(round(time()-t0, 2)))

INFO:root:Initializing SKIPGRAM...
INFO:root:number of graphs: 344
INFO:root:subgraph vocabulary size: 3804
INFO:root:total number of subgraphs to be trained: 34837
INFO:root:Epoch: 0 : Average loss for step: 100 : 44.053448
INFO:root:Epoch: 0 : Average loss for step: 200 : 37.702990
INFO:root:#########################   Epoch: 0 :  35.445183, 1.09 sec.  #####################
INFO:root:Epoch: 1 : Average loss for step: 100 : 25.386242
INFO:root:Epoch: 1 : Average loss for step: 200 : 24.069743
INFO:root:#########################   Epoch: 1 :  22.902555, 0.97 sec.  #####################
INFO:root:Epoch: 2 : Average loss for step: 100 : 15.687668
INFO:root:Epoch: 2 : Average loss for step: 200 : 16.182734
INFO:root:#########################   Epoch: 2 :  15.780944, 0.97 sec.  #####################
INFO:root:Epoch: 3 : Average loss for step: 100 : 13.728236
INFO:root:Epoch: 3 : Average loss for step: 200 : 12.342629
INFO:root:#########################   Epoch: 3 :  12.033786, 0.96 sec.  #

INFO:root:Epoch: 38 : Average loss for step: 100 : 1.192453
INFO:root:Epoch: 38 : Average loss for step: 200 : 1.208227
INFO:root:#########################   Epoch: 38 :  1.186123, 0.95 sec.  #####################
INFO:root:Epoch: 39 : Average loss for step: 100 : 1.231534
INFO:root:Epoch: 39 : Average loss for step: 200 : 1.195807
INFO:root:#########################   Epoch: 39 :  1.178413, 0.94 sec.  #####################
INFO:root:Epoch: 40 : Average loss for step: 100 : 1.222653
INFO:root:Epoch: 40 : Average loss for step: 200 : 1.195820
INFO:root:#########################   Epoch: 40 :  1.207544, 0.95 sec.  #####################
INFO:root:Epoch: 41 : Average loss for step: 100 : 1.234114
INFO:root:Epoch: 41 : Average loss for step: 200 : 1.215157
INFO:root:#########################   Epoch: 41 :  1.199390, 0.94 sec.  #####################
INFO:root:Epoch: 42 : Average loss for step: 100 : 1.153730
INFO:root:Epoch: 42 : Average loss for step: 200 : 1.154644
INFO:root:##############

INFO:root:#########################   Epoch: 76 :  0.919826, 0.94 sec.  #####################
INFO:root:Epoch: 77 : Average loss for step: 100 : 0.904998
INFO:root:Epoch: 77 : Average loss for step: 200 : 0.923373
INFO:root:#########################   Epoch: 77 :  0.934361, 0.93 sec.  #####################
INFO:root:Epoch: 78 : Average loss for step: 100 : 0.918342
INFO:root:Epoch: 78 : Average loss for step: 200 : 0.922286
INFO:root:#########################   Epoch: 78 :  0.915841, 0.93 sec.  #####################
INFO:root:Epoch: 79 : Average loss for step: 100 : 0.942286
INFO:root:Epoch: 79 : Average loss for step: 200 : 0.933504
INFO:root:#########################   Epoch: 79 :  0.910514, 0.93 sec.  #####################
INFO:root:Epoch: 80 : Average loss for step: 100 : 0.955701
INFO:root:Epoch: 80 : Average loss for step: 200 : 0.930187
INFO:root:#########################   Epoch: 80 :  0.925473, 0.94 sec.  #####################
INFO:root:Epoch: 81 : Average loss for step: 100 :

In [24]:
perform_classification (corpus_dir, wl_extn, embedding_fname, class_labels_fname)

INFO:root:Y (label) matrix shape: (344,)
INFO:root:Train and Test matrix shapes: (309, 1024), (35, 1024), (309,), (35,) 


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    8.8s finished
INFO:root:best classifier model's hyperparamters
INFO:root:Linear SVM accuracy: 0.4857142857142857
INFO:root:             precision    recall  f1-score   support

         -1       0.40      0.57      0.47        14
          1       0.60      0.43      0.50        21

avg / total       0.52      0.49      0.49        35



# To run many time

In [52]:
# These parts need not be repeated

####################### set parametrs #######################################
corpus_dir = "/home/ipsita/BTP/graph2vec/data/kdd_datasets/proteins"
output_dir = "/home/ipsita/BTP/graph2vec/embeddings"
batch_size = 128
epochs = 1000
embedding_size = 1024
num_negsample = 10
learning_rate = 0.3
wlk_h = 3
label_filed_name = 'Label'
class_labels_fname = '/home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc.Labels'

wl_extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

assert os.path.exists(corpus_dir), "File {} does not exist".format(corpus_dir)
assert os.path.exists(output_dir), "Dir {} does not exist".format(output_dir)

######################## load graphs from corpus ###############################
graph_files = get_files(dirname=corpus_dir, extn='.gexf', max_files=0)
logging.info('Loaded {} graph file names form {}'.format(len(graph_files),corpus_dir))

######################## make corpus files #####################################
t0 = time.time()
wlk_relabel_and_dump_memory_version(graph_files, max_h=wlk_h, node_label_attr_name=label_filed_name)
logging.info('dumped sg2vec sentences in {} sec.'.format(time.time() - t0))

INFO:root:Loaded 1113 graph file names form /home/ipsita/BTP/graph2vec/data/kdd_datasets/proteins


loaded all graphs in 3.78 sec
initial relabeling done in 0.39 sec
WL iteration 1 done in 0.43 sec.
num of WL rooted subgraphs in iter 1 is 297
WL iteration 2 done in 0.51 sec.
num of WL rooted subgraphs in iter 2 is 20962
WL iteration 3 done in 0.43 sec.
num of WL rooted subgraphs in iter 3 is 35676


INFO:root:dumped sg2vec sentences in 8.048662900924683 sec.


dumped sg2vec sentences in 2.47


In [54]:
total_acc = []
fname = "/home/ipsita/BTP/graph2vec/embeddings/proteins_dims_1024_epochs_1000_lr_0.3_embeddings.txt"

for i in range(0, 5):
    
    if os.path.isfile(fname):
        os.remove(fname);

    ######################### training ##############################################
    t0 = time.time()
    embedding_fname = train_skipgram(corpus_dir, wl_extn, learning_rate, embedding_size, num_negsample,
                                     epochs, batch_size, output_dir)
    logging.info('Trained the skipgram model in {} sec.'.format(round(time.time()-t0, 2)))
    
    ######################## classify ###############################################
    a = perform_classification (corpus_dir, wl_extn, embedding_fname, class_labels_fname)
    print('i=',i,'accuracy=',a)
    total_acc.append(a)
    

INFO:root:Initializing SKIPGRAM...
INFO:root:number of graphs: 1113
INFO:root:subgraph vocabulary size: 56940
INFO:root:total number of subgraphs to be trained: 173885
INFO:root:Epoch: 0 : Average loss for step: 100 : 79.779760
INFO:root:Epoch: 0 : Average loss for step: 200 : 78.744366
INFO:root:Epoch: 0 : Average loss for step: 300 : 77.429300
INFO:root:Epoch: 0 : Average loss for step: 400 : 76.574866
INFO:root:Epoch: 0 : Average loss for step: 500 : 75.715008
INFO:root:Epoch: 0 : Average loss for step: 600 : 75.532381
INFO:root:Epoch: 0 : Average loss for step: 700 : 74.627882
INFO:root:Epoch: 0 : Average loss for step: 800 : 73.963933
INFO:root:Epoch: 0 : Average loss for step: 900 : 73.242513
INFO:root:Epoch: 0 : Average loss for step: 1000 : 72.729555
INFO:root:Epoch: 0 : Average loss for step: 1100 : 71.831074
INFO:root:Epoch: 0 : Average loss for step: 1200 : 71.223213
INFO:root:Epoch: 0 : Average loss for step: 1300 : 70.500670
INFO:root:#########################   Epoch: 0 :

INFO:root:Epoch: 9 : Average loss for step: 400 : 14.947213
INFO:root:Epoch: 9 : Average loss for step: 500 : 15.068150
INFO:root:Epoch: 9 : Average loss for step: 600 : 14.912479
INFO:root:Epoch: 9 : Average loss for step: 700 : 15.102770
INFO:root:Epoch: 9 : Average loss for step: 800 : 14.895819
INFO:root:Epoch: 9 : Average loss for step: 900 : 14.787950
INFO:root:Epoch: 9 : Average loss for step: 1000 : 14.698368
INFO:root:Epoch: 9 : Average loss for step: 1100 : 14.783929
INFO:root:Epoch: 9 : Average loss for step: 1200 : 14.707936
INFO:root:Epoch: 9 : Average loss for step: 1300 : 14.711246
INFO:root:#########################   Epoch: 9 :  14.601772, 4.82 sec.  #####################
INFO:root:Epoch: 10 : Average loss for step: 100 : 14.281201
INFO:root:Epoch: 10 : Average loss for step: 200 : 13.704992
INFO:root:Epoch: 10 : Average loss for step: 300 : 13.539237
INFO:root:Epoch: 10 : Average loss for step: 400 : 13.309503
INFO:root:Epoch: 10 : Average loss for step: 500 : 13.3465

INFO:root:Epoch: 18 : Average loss for step: 900 : 4.882539
INFO:root:Epoch: 18 : Average loss for step: 1000 : 4.917760
INFO:root:Epoch: 18 : Average loss for step: 1100 : 4.902273
INFO:root:Epoch: 18 : Average loss for step: 1200 : 4.878293
INFO:root:Epoch: 18 : Average loss for step: 1300 : 4.869948
INFO:root:#########################   Epoch: 18 :  4.823466, 4.82 sec.  #####################
INFO:root:Epoch: 19 : Average loss for step: 100 : 4.232739
INFO:root:Epoch: 19 : Average loss for step: 200 : 4.578248
INFO:root:Epoch: 19 : Average loss for step: 300 : 4.640207
INFO:root:Epoch: 19 : Average loss for step: 400 : 4.479966
INFO:root:Epoch: 19 : Average loss for step: 500 : 4.566091
INFO:root:Epoch: 19 : Average loss for step: 600 : 4.545211
INFO:root:Epoch: 19 : Average loss for step: 700 : 4.549294
INFO:root:Epoch: 19 : Average loss for step: 800 : 4.427880
INFO:root:Epoch: 19 : Average loss for step: 900 : 4.385632
INFO:root:Epoch: 19 : Average loss for step: 1000 : 4.373868
I

INFO:root:#########################   Epoch: 27 :  2.468945, 4.96 sec.  #####################
INFO:root:Epoch: 28 : Average loss for step: 100 : 2.460936
INFO:root:Epoch: 28 : Average loss for step: 200 : 2.403476
INFO:root:Epoch: 28 : Average loss for step: 300 : 2.342726
INFO:root:Epoch: 28 : Average loss for step: 400 : 2.348811
INFO:root:Epoch: 28 : Average loss for step: 500 : 2.329344
INFO:root:Epoch: 28 : Average loss for step: 600 : 2.389444
INFO:root:Epoch: 28 : Average loss for step: 700 : 2.399916
INFO:root:Epoch: 28 : Average loss for step: 800 : 2.400550
INFO:root:Epoch: 28 : Average loss for step: 900 : 2.439070
INFO:root:Epoch: 28 : Average loss for step: 1000 : 2.433769
INFO:root:Epoch: 28 : Average loss for step: 1100 : 2.467071
INFO:root:Epoch: 28 : Average loss for step: 1200 : 2.466007
INFO:root:Epoch: 28 : Average loss for step: 1300 : 2.474010
INFO:root:#########################   Epoch: 28 :  2.470569, 4.87 sec.  #####################
INFO:root:Epoch: 29 : Averag

INFO:root:Epoch: 37 : Average loss for step: 500 : 1.811485
INFO:root:Epoch: 37 : Average loss for step: 600 : 1.792718
INFO:root:Epoch: 37 : Average loss for step: 700 : 1.797297
INFO:root:Epoch: 37 : Average loss for step: 800 : 1.815020
INFO:root:Epoch: 37 : Average loss for step: 900 : 1.837462
INFO:root:Epoch: 37 : Average loss for step: 1000 : 1.834645
INFO:root:Epoch: 37 : Average loss for step: 1100 : 1.840804
INFO:root:Epoch: 37 : Average loss for step: 1200 : 1.836000
INFO:root:Epoch: 37 : Average loss for step: 1300 : 1.826663
INFO:root:#########################   Epoch: 37 :  1.833542, 4.87 sec.  #####################
INFO:root:Epoch: 38 : Average loss for step: 100 : 1.756670
INFO:root:Epoch: 38 : Average loss for step: 200 : 1.757758
INFO:root:Epoch: 38 : Average loss for step: 300 : 1.762160
INFO:root:Epoch: 38 : Average loss for step: 400 : 1.793712
INFO:root:Epoch: 38 : Average loss for step: 500 : 1.822654
INFO:root:Epoch: 38 : Average loss for step: 600 : 1.834338
IN

...

INFO:root:#########################   Epoch: 968 :  0.428130, 4.83 sec.  #####################
INFO:root:Epoch: 969 : Average loss for step: 100 : 0.474541
INFO:root:Epoch: 969 : Average loss for step: 200 : 0.448146
INFO:root:Epoch: 969 : Average loss for step: 300 : 0.429450
INFO:root:Epoch: 969 : Average loss for step: 400 : 0.424299
INFO:root:Epoch: 969 : Average loss for step: 500 : 0.418522
INFO:root:Epoch: 969 : Average loss for step: 600 : 0.420373
INFO:root:Epoch: 969 : Average loss for step: 700 : 0.407746
INFO:root:Epoch: 969 : Average loss for step: 800 : 0.406742
INFO:root:Epoch: 969 : Average loss for step: 900 : 0.428597
INFO:root:Epoch: 969 : Average loss for step: 1000 : 0.423573
INFO:root:Epoch: 969 : Average loss for step: 1100 : 0.420024
INFO:root:Epoch: 969 : Average loss for step: 1200 : 0.418709
INFO:root:Epoch: 969 : Average loss for step: 1300 : 0.418176
INFO:root:#########################   Epoch: 969 :  0.421282, 4.81 sec.  #####################
INFO:root:Epo

INFO:root:Epoch: 978 : Average loss for step: 300 : 0.434891
INFO:root:Epoch: 978 : Average loss for step: 400 : 0.418604
INFO:root:Epoch: 978 : Average loss for step: 500 : 0.418682
INFO:root:Epoch: 978 : Average loss for step: 600 : 0.410736
INFO:root:Epoch: 978 : Average loss for step: 700 : 0.403139
INFO:root:Epoch: 978 : Average loss for step: 800 : 0.397463
INFO:root:Epoch: 978 : Average loss for step: 900 : 0.396312
INFO:root:Epoch: 978 : Average loss for step: 1000 : 0.400666
INFO:root:Epoch: 978 : Average loss for step: 1100 : 0.401504
INFO:root:Epoch: 978 : Average loss for step: 1200 : 0.399487
INFO:root:Epoch: 978 : Average loss for step: 1300 : 0.400465
INFO:root:#########################   Epoch: 978 :  0.401505, 4.81 sec.  #####################
INFO:root:Epoch: 979 : Average loss for step: 100 : 0.418461
INFO:root:Epoch: 979 : Average loss for step: 200 : 0.429870
INFO:root:Epoch: 979 : Average loss for step: 300 : 0.402908
INFO:root:Epoch: 979 : Average loss for step: 4

INFO:root:Epoch: 987 : Average loss for step: 600 : 0.403952
INFO:root:Epoch: 987 : Average loss for step: 700 : 0.405999
INFO:root:Epoch: 987 : Average loss for step: 800 : 0.422188
INFO:root:Epoch: 987 : Average loss for step: 900 : 0.423905
INFO:root:Epoch: 987 : Average loss for step: 1000 : 0.420260
INFO:root:Epoch: 987 : Average loss for step: 1100 : 0.420815
INFO:root:Epoch: 987 : Average loss for step: 1200 : 0.422855
INFO:root:Epoch: 987 : Average loss for step: 1300 : 0.420051
INFO:root:#########################   Epoch: 987 :  0.419027, 6.46 sec.  #####################
INFO:root:Epoch: 988 : Average loss for step: 100 : 0.409939
INFO:root:Epoch: 988 : Average loss for step: 200 : 0.448515
INFO:root:Epoch: 988 : Average loss for step: 300 : 0.416295
INFO:root:Epoch: 988 : Average loss for step: 400 : 0.400403
INFO:root:Epoch: 988 : Average loss for step: 500 : 0.403945
INFO:root:Epoch: 988 : Average loss for step: 600 : 0.384035
INFO:root:Epoch: 988 : Average loss for step: 7

INFO:root:Epoch: 996 : Average loss for step: 900 : 0.419082
INFO:root:Epoch: 996 : Average loss for step: 1000 : 0.413965
INFO:root:Epoch: 996 : Average loss for step: 1100 : 0.410504
INFO:root:Epoch: 996 : Average loss for step: 1200 : 0.410839
INFO:root:Epoch: 996 : Average loss for step: 1300 : 0.406224
INFO:root:#########################   Epoch: 996 :  0.404665, 4.80 sec.  #####################
INFO:root:Epoch: 997 : Average loss for step: 100 : 0.399301
INFO:root:Epoch: 997 : Average loss for step: 200 : 0.408877
INFO:root:Epoch: 997 : Average loss for step: 300 : 0.414717
INFO:root:Epoch: 997 : Average loss for step: 400 : 0.426734
INFO:root:Epoch: 997 : Average loss for step: 500 : 0.414308
INFO:root:Epoch: 997 : Average loss for step: 600 : 0.406165
INFO:root:Epoch: 997 : Average loss for step: 700 : 0.407086
INFO:root:Epoch: 997 : Average loss for step: 800 : 0.397095
INFO:root:Epoch: 997 : Average loss for step: 900 : 0.392819
INFO:root:Epoch: 997 : Average loss for step: 1

KeyError: '1000'

In [55]:
total_acc

[]

In [49]:
import argparse,os,logging,psutil,time
from joblib import Parallel,delayed

from utils import get_files
from train_utils import train_skipgram
from classify import perform_classification
from make_graph2vec_corpus import *
from time import time

logger = logging.getLogger()
logger.setLevel("INFO")


def main(args):
    
    '''
    :param args: arguments for
    1. training the skigram model for learning subgraph representations
    2. construct the deep WL kernel using the learnt subgraph representations
    3. performing graph classification using  the WL and deep WL kernel
    :return: None
    '''
    corpus_dir = args.corpus
    output_dir = args.output_dir
    batch_size = args.batch_size
    epochs = args.epochs
    embedding_size = args.embedding_size
    num_negsample = args.num_negsample
    learning_rate = args.learning_rate
    wlk_h = args.wlk_h
    label_filed_name = args.label_filed_name
    class_labels_fname = args.class_labels_file_name

    wl_extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

    assert os.path.exists(corpus_dir), "File {} does not exist".format(corpus_dir)
    assert os.path.exists(output_dir), "Dir {} does not exist".format(output_dir)

    graph_files = get_files(dirname=corpus_dir, extn='.gexf', max_files=0)
    logging.info('Loaded {} graph file names form {}'.format(len(graph_files),corpus_dir))


    t0 = time()
    wlk_relabel_and_dump_memory_version(graph_files, max_h=wlk_h, node_label_attr_name=label_filed_name)
    logging.info('dumped sg2vec sentences in {} sec.'.format(time() - t0))

    t0 = time()
    embedding_fname = train_skipgram(corpus_dir, wl_extn, learning_rate, embedding_size, num_negsample,
                                     epochs, batch_size, output_dir)
    logging.info('Trained the skipgram model in {} sec.'.format(round(time()-t0, 2)))

    perform_classification (corpus_dir, wl_extn, embedding_fname, class_labels_fname)

ModuleNotFoundError: No module named 'utils'